In [ ]:
#@markdown ###Driveのマウント
#@markdown 自分が作ったときはDiriveをマウントしてcsvをロードした
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#@markdown ###gitクローンでcsvをロード(2020/05/24)
#@markdown 共有者も実行できるように追加した
!git clone https://github.com/tomoe6/Idol_type_Cheaker.git

fatal: destination path 'Idol_type_Cheaker' already exists and is not an empty directory.


In [ ]:
#@markdown ###csvファイル読み込み　 　 
#@markdown　> load_data[] -- [Type, ひらがな]
load_data = [] #[Type, ひらがな] 
#旧path = "/content/drive/My Drive/PythonMachineLearning/Chainer/Idol.csv"
path = "/content/Idol_type_Cheaker/Idol.csv"
with open(path) as f:
    for s_line in f:
      load_data.append(s_line.split(",")[:3:2])

In [ ]:
#@markdown ###hiragana関数定義
#@markdown >dic[] -- [ひらがな]  
#@markdown >hiragana(s: str) --文字列を数字対応表に従いlist型数列を返す
dic = []
def hiragana(s):
  ret = []
  for i in s:
    if i in dic:
      ret.append(dic.index(i))
    else:
      dic.append(i)
      ret.append(dic.index(i))
  return ret

In [ ]:
#@markdown ###hiragana関数を用いてアイドル名を数列をにする
#@markdown >class_data -- アイドルタイプを数字でもつ --Cu: 0, Co: 1, Pa: 2  
#@markdown >number_data --アイドル名を数字でもつ  --hiragana()に従う
number_data = []
class_data = []
count = 0
for i in load_data:
  number_data.append(hiragana(i[1]))
  if i[0] == "Cu":
    count+=1
    label = 0
  elif i[0] == "Co":
    count +=1
    label = 1
  elif i[0] == "Pa":
    count +=1
    label = 2
  class_data.append(label)
#print(len(number_data),number_data)
#print(len(class_data),class_data)
#print(count == len(class_data))

In [ ]:
#@markdown ###number_dataからarraylistを作成
#@markdown >np_data[len(number_data),最大文字数,日本語の数] --数字のindexに1をもつ
import numpy as np
np_data = np.zeros((190,15,81), dtype = "float32")
for i,datas in enumerate(number_data):
  for k,data in enumerate(datas):
    np_data[i,k,data] = 1

In [ ]:
#@markdown ###実際に学習に用いるX,Yを作成
#@markdown X --((アイドル数,最大文字数*日本語の数),dtype = "float32")
X = np.zeros((190,1215), dtype = "float32")
for i in range(190):
  X[i] = np_data[i].flatten()
#@markdown Y --class_dataをint32,arrayに変換したもの
Y = np.array(class_data, dtype = "int32")

In [ ]:
Y.shape

(190,)

In [ ]:
X.shape

(190, 1215)

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

class MLP(chainer.Chain):
  def __init__(self, n_unis = 1216, n_out = 3):
    super(MLP, self).__init__(
      l1 = L.Linear(1215, n_unis),
      l2 = L.Linear(n_unis, n_unis),
      l3 = L.Linear(n_unis, n_out)
    )
  def __call__(self, x):
    h1= F.relu(self.l1(x))
    h2 = F.relu(self.l2(h1))
    return self.l3(h2)

In [ ]:
from chainer import training, datasets
train ,test= datasets.split_dataset_random(chainer.datasets.TupleDataset(X,Y),180)
train_iter = chainer.iterators.SerialIterator(train, 10)
test_iter = chainer.iterators.SerialIterator(test, 1,repeat=False, shuffle=False)

In [ ]:
model = L.Classifier(MLP())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (50, 'epoch'), out="result")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
trainer.extend(extensions.ProgressBar())

In [ ]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.10715     1.11379               0.361111       0.4                       
2           0.667187    1.37178               0.833333       0.3                       
3           0.151149    1.95337               0.977778       0.4                       
4           0.0230129   1.86153               1              0.4                       
5           0.00423454  1.96859               1              0.5                       
     total [#####.............................................] 11.11%
this epoch [###########################.......................] 55.56%
       100 iter, 5 epoch / 50 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
6           0.00118135  2.00327               1              0.4                       
7           0.000810185  2.06239               1              0.4                       
8           0.00062521  2.0992                1              

In [ ]:
#@title アイドルタイプ判別機
#@markdown 下に判別したい名前を入力して実行
idol_name = "\u3086\u304D\u306F" #@param {type:"string"}
sibuya = hiragana(idol_name)
sibuya_date = np.zeros((15,81), dtype = "float32")

for k,date in enumerate(sibuya):
   sibuya_date[k,date] = 1
sibuya_date.flatten().shape
y = model.predictor(np.array([sibuya_date]).astype(np.float32))
print(F.softmax(y).data)
print(F.softmax(y).data.argmax(axis=1))
res = F.softmax(y).data.argmax(axis=1)[0]
if(res == 0): type_idol = "Cu"
elif res == 1: type_idol = "Co"
else : type_idol = "Pa"
print("「"+idol_name+"」は"+type_idol+"です")
#0Cu 1Co 2 Pa
#@markdown Cu Co Pa　を名前から判断します


[[0.02275672 0.32882094 0.64842236]]
[2]
「ゆきは」はPaです
